In [ ]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

config = tf.ConfigProto()
config.gpu_options.visible_device_list='0'
config.gpu_options.per_process_gpu_memory_fraction = 0.25
set_session(tf.Session(config=config))

In [ ]:
import numpy as np 
signal_data = np.cos(np.arange(1600)*(20*np.pi/1000))[:,None]

In [ ]:
import matplotlib.pyplot as plt

plot_x = np.arange(1600)
plot_y = signal_data
plt.plot(plot_x, plot_y)
plt.show()

In [ ]:
## create dataset from generated data
def create_dataset(signal_data, look_back=1):
    dataX, dataY = [], []
    for i in range(len(signal_data) - look_back):
        dataX.append(signal_data[i:(i+look_back), 0])
        dataY.append(signal_data[i + look_back, 0])
    return np.array(dataX), np.array(dataY)



In [ ]:
## -1.0~1.0 --> 0.0~1.0  set look_back = 40 
from sklearn.preprocessing import MinMaxScaler
look_back = 40 

# data preprocessing
scaler = MinMaxScaler(feature_range=(0,1))
signal_data = scaler.fit_transform(signal_data)

# data split
train = signal_data[0:800]
val = signal_data[800:1200]
test = signal_data[1200:]

# dataset creation
x_train, y_train = create_dataset(train, look_back)
x_val, y_val = create_dataset(val, look_back)
x_test, y_test = create_dataset(test, look_back)

In [ ]:
# dataset preprocessing
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_val = np.reshape(x_val, (x_val.shape[0], x_val.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

x_train = np.squeeze(x_train) 
x_val = np.squeeze(x_val)
x_test = np.squeeze(x_test)



In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

# model design
model = Sequential()
model.add(Dense(32, input_dim=40, activation='relu'))
model.add(Dropout(0.3)) 
for i in range(2):
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.3))
model.add(Dense(1))

In [ ]:
model.compile(loss='mean_squared_error', optimizer='adagrad')
hist = model.fit(x_train, y_train, epochs=200, batch_size=32, validation_data=(x_val, y_val))

In [ ]:
# check train 
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.ylim(0.0, 0.15)
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
trainScore = model.evaluate(x_train, y_train, verbose=0)
print('Train Score: ', trainScore)
valScore = model.evaluate(x_val, y_val, verbose=0)
print('Validation Score: ', valScore)
testScore = model.evaluate(x_test, y_test, verbose=0)
print('Test Score:', testScore) 


In [ ]:
look_ahead = 250
xhat = x_test[0, None]
predictions = np.zeros((look_ahead, 1))
for i in range(look_ahead):     
    prediction = model.predict(xhat, batch_size=32)
    predictions[i] = prediction 
    xhat = np.hstack([xhat[:,1:], prediction])
    
    
plt.figure(figsize=(12, 5))
plt.plot(np.arange(look_ahead), predictions, 'r', label='prediction')
plt.plot(np.arange(look_ahead), y_test[:look_ahead], label='test function')
plt.legend()
plt.show()